In [1]:
import pandas as pd
import re
import openai
import re
import requests
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity

API_KEY = "6e78d72911ba4064803b8b14e1f272b8"
RESOURCE_ENDPOINT = "https://azopenaidemo.openai.azure.com/" 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
#Note: The openai-python library support for Azure OpenAI is in preview.
openai.api_version = "2023-05-15"



### Data generation (Skip this step if data is already generated )

In [26]:
user_message = ""
def generate_titles():
    user_message =f""" 
    generate 100 titles of customer support articles in your industry, focusing on the areas of payroll and HR that support agent can look up to answer questions from customers who are employees with payroll managed by ADP.
    Output data into a list 
 
"""

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a customer support agent for ADP company"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

titles = generate_titles()

In [27]:
titles = titles.split("\n")

In [28]:
titles[:10]

['1. Understanding Your ADP Pay Statement',
 '2. How to Enroll in ADP Direct Deposit',
 '3. Updating Your Tax Information with ADP',
 '4. How to Register for ADP Employee Self Service',
 '5. What to Do When Your Paycheck is Late',
 '6. How to Access Payroll Reports in ADP',
 '7. Understanding Your ADP W-2 Form',
 '8. How to Change Your ADP Password',
 '9. Adding or Removing Dependents in ADP',
 '10. How to View Your Pay Stub in ADP']

In [29]:
import time
def generate_article(title):
    user_message =f""" 
    given this title of an article "{title}" that support agents can look up to answer questions from customers who are employees with payroll managed by ADP.
    Generate detail content of the article 
 
"""
    i=0
    while i<5:
        try:
            response = openai.ChatCompletion.create(
                engine="gpt-35-turbo", # engine = "deployment_name".
                messages=[
                    {"role": "system", "content": "You are a customer support agent for ADP company"},
                    {"role": "user", "content":user_message },
                ]
            )
            return response['choices'][0]['message']['content']
        except:
            i+=1
            time.sleep(5)
            

contents =[]
for title in titles:
    contents.append(generate_article(title))

In [30]:
import os
folder_path ="../../../data/agent_assistant"
os.makedirs(folder_path, exist_ok=True)
i=0
articles ={}
for title, content in zip(titles, contents):
    file_name = f"article_{i}"
    file_content = title+"\n\n"+content

    with open(os.path.join(folder_path,file_name), 'w') as file:
        file.write(file_content)

    articles[file_name]= file_content
    i+=1



#### Document Chunking Approach

In [4]:
MAX_SECTION_LENGTH = 1000
SENTENCE_SEARCH_LIMIT = 100
SECTION_OVERLAP = 100
verbose= True
def split_text(all_text):
    SENTENCE_ENDINGS = [".", "!", "?"]
    WORDS_BREAKS = [",", ";", ":", " ", "(", ")", "[", "]", "{", "}", "\t", "\n"]
    # if verbose: print(f"Splitting '{filename}' into sections")

    # def find_page(offset):
    #     l = len(page_map)
    #     for i in range(l - 1):
    #         if offset >= page_map[i][1] and offset < page_map[i + 1][1]:
    #             return i
    #     return l - 1

    # all_text = "".join(p[2] for p in page_map)
    length = len(all_text)
    start = 0
    end = length
    chunk=0
    while start + SECTION_OVERLAP < length:
        last_word = -1
        end = start + MAX_SECTION_LENGTH

        if end > length:
            end = length
        else:
            # Try to find the end of the sentence
            while end < length and (end - start - MAX_SECTION_LENGTH) < SENTENCE_SEARCH_LIMIT and all_text[end] not in SENTENCE_ENDINGS:
                if all_text[end] in WORDS_BREAKS:
                    last_word = end
                end += 1
            if end < length and all_text[end] not in SENTENCE_ENDINGS and last_word > 0:
                end = last_word # Fall back to at least keeping a whole word
        if end < length:
            end += 1

        # Try to find the start of the sentence or at least a whole word boundary
        last_word = -1
        while start > 0 and start > end - MAX_SECTION_LENGTH - 2 * SENTENCE_SEARCH_LIMIT and all_text[start] not in SENTENCE_ENDINGS:
            if all_text[start] in WORDS_BREAKS:
                last_word = start
            start -= 1
        if all_text[start] not in SENTENCE_ENDINGS and last_word > 0:
            start = last_word
        if start > 0:
            start += 1

        section_text = all_text[start:end]
        chunk += 1
        yield (section_text, chunk)

        start = end - SECTION_OVERLAP
        
    if start + SECTION_OVERLAP < end:
        yield (all_text[start:end], chunk)


### Enriching article chunks with Summary

In [25]:
# openai.api_version = "2023-05-15"
# import json
# import time
# gpt_turbo_model = "gpt-35-turbo"
# gpt_4_32k_model = "" #find name of gpt_4_32k model

# def enrich_chunk(chunk_content):
#     user_message=f""" 
#     Given the document below, extract the main topic and a short summary of the content
#     {{"topic": "Topic Name", "summary": "short summary of content"}}
#     Just output data, do not add any comment.
#     <<document>>
#     {chunk_content}
#     <<document>>
#     """
    

#     system_message = """
#     You are an AI assistant that helps extract information from text. 
#     """
#     gpt_model= gpt_turbo_model #default choice, it can be better to go with GPT-4
#     i=0
#     output=""
#     while i<10: #if the output format is not as expected or if there's a throttling error, retry up to 10 times.
#         try:
#             response = openai.ChatCompletion.create(
#                 engine=gpt_model, 
#                 messages=[
#                     {"role": "system", "content": system_message},
#                     {"role": "user", "content":user_message },
#                 ]
#             )
#             response=response['choices'][0]['message']['content']
#             output = json.loads(response)
#             output["topic"],output["summary"] #this is just to validate the format
#             break
#         except Exception as e:
#             print("temporary exception occured, will retry after 3s", str(e))
#             i+=1
#             time.sleep(3)
#     if len("output")==0:
#         raise Exception("Cannot extract the content after retrying 10 times")
#     return output
# enriched_chunked_articles=[]
# for article in enriched_articles:
#     article_file = article["article_file"]
#     article_content = article["article_content"]
#     enriched_article_chunks =[]
#     for chunk_content, chunk_number in split_text(article_content):
#         enriched_chunk = enrich_chunk(chunk_content)
#         enriched_chunk['chunk_content'] = chunk_content
#         enriched_article_chunks.append(enriched_chunk)


#     enriched_chunked_articles.append((article_file,article_content, enriched_article_chunks))

Building map for topic

In [26]:
# openai.api_version = "2022-12-01"
# topic_content={}
# for enriched_article in enriched_chunked_articles:
#     chunks = enriched_article[2]
#     for chunk in chunks:
#         article_topic_id= enriched_article[0]+"###"+ chunk['topic']
#         topic_content[article_topic_id]=chunk['chunk_content']
        

#### Simple chunking without enrichment


In [22]:
openai.api_version = "2023-05-15"
import json
import time
chunked_articles=[]
for article in enriched_articles:
    article_file = article["article_file"]
    article_content = article["article_content"]
    article_chunks =[]
    for chunk_content, chunk_number in split_text(article_content):
        article_chunks.append(chunk_content)
    chunked_articles.append((article_file,article_content, article_chunks))
openai.api_version = "2022-12-01"
chunks_emb=[]
for article in chunked_articles:
    chunks = article[2]
    chunk_no =0
    for chunk in chunks:
        chunk_id= article[0]+"###"+ str(chunk_no)
        chunk_emb = get_embedding(chunk, engine = 'text-embedding-ada-002')
        chunks_emb.append((chunk_id, chunk, chunk_emb))
        

In [24]:
import json
folder_path ="../../data/agent_assistant"

with open(os.path.join(folder_path,"chunk_emb_map.json"), "w") as fp:
    json.dump(chunks_emb,fp) 



#### Semantic Chunking and Enrichment with GPT
To help semantic search better, we can run GPT models (ChatGPT, GPT-4, GPT-4-32k) through knowledge articles to create topics out of each document together with original content. Basically, the idea is to break the raw article content into topics with clear description so that search can better map question/query with right document


In [31]:
openai.api_version = "2023-05-15"
import json
import time
gpt_turbo_model = "gpt-35-turbo"
gpt_4_32k_model = "" #find name of gpt_4_32k model

def enrich(article_content):
    user_message=f""" 
    Given the document below, extract key topics, each with a short description and corresponding extracted original content from the document. 
    Output data in a list with this format:
    [{{"topic": "Topic Name", "description": "description of topic", "extracted_content": "original content extracted from the document"}}]
    Just output data, do not add any comment.
    <<document>>
    {article_content}
    <<document>>
    """
    

    system_message = """
    You are an AI assistant that helps people organize data. 
    """
    gpt_model= gpt_turbo_model #default choice, it can be better to go with GPT-4
    if len(article_content)>14500: #if the count of document character > 2x the limit of GPT_turbo, go with GPT-4-32K-model
        gpt_model= gpt_4_32k_model
        print(f"using gpt-4-32k model named {gpt_model} for this enrichment")

    i=0
    output=""
    while i<10: #if the output format is not as expected or if there's a throttling error, retry up to 10 times.
        try:
            response = openai.ChatCompletion.create(
                engine=gpt_model, 
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content":user_message },
                ]
            )
            response=response['choices'][0]['message']['content']
            output = json.loads(response)
            for content in output:
                content["topic"],content["description"],content["extracted_content"] #this is just to validate the format
            break
        except Exception as e:
            print("temporary exception occured, will retry after 3s", str(e))
            i+=1
            time.sleep(3)
    if len("output")==0:
        raise Exception("Cannot extract the content after retrying 10 times")
    return output
enriched_articles =[]
for article_file, article_content in articles.items():
    enriched_article ={}
    try:
        enriched_article["enriched_content"] = enrich(article_content)
    except Exception:
        print(f"Warning, document {article_file} cannot be enriched and is not included in the result, please check")
    enriched_article["article_file"] = article_file
    enriched_article["article_content"] = article_content

    enriched_articles.append(enriched_article)



temporary exception occured, will retry after 3s Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
temporary exception occured, will retry after 3s Expecting ',' delimiter: line 3 column 459 (char 586)
temporary exception occured, will retry after 3s Expecting ',' delimiter: line 1 column 1343 (char 1342)
temporary exception occured, will retry after 3s Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
temporary exception occured, will retry after 3s The operation was timeout. { "error": { "code": "Timeout", "message": "The operation was timeout." } } 408 {'error': {'code': 'Timeout', 'message': 'The operation was timeout.'}} {'Content-Length': '75', 'Content-Type': 'application/json', 'apim-request-id': 'a87746f9-8d52-4867-a1e1-52b8da28715e', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'East US', 'Date': 'Mon, 22 May 2023 21:28:00 GMT'}
temporary exc

#### Building content map for topic 

In [ ]:
openai.api_version = "2022-12-01"
topic_content={}
for enriched_article in enriched_articles:
    topics = enriched_article['enriched_content']
    for topic in topics:
        article_topic_id= enriched_article['article_file']+"###"+ topic['topic']
        topic_content[article_topic_id]=topic['extracted_content']
        

### Generate embeddings

In [ ]:
openai.api_version = "2022-12-01"
enriched_emb={}
for enriched_article in enriched_articles:
    topics = enriched_article['enriched_content']
    for topic in topics:
        article_topic_id= enriched_article['article_file']+"###"+ topic['topic']
        topic_emb =get_embedding(topic['topic']+"\n"+topic['description'], engine = 'text-embedding-ada-002')
        content_emb = get_embedding(topic['topic']+"\n"+topic['description']+"\n"+topic['extracted_content'], engine = 'text-embedding-ada-002')
        enriched_emb[article_topic_id]=(topic_emb,content_emb)
        

### Persist data

In [ ]:
import json
folder_path ="../../data/agent_assistant"

with open(os.path.join(folder_path,"enriched_emb.json"), "w") as fp:
    json.dump(enriched_emb,fp) 

with open(os.path.join(folder_path,"enriched_articles.json"), "w") as fp:
    json.dump(enriched_articles,fp) 
with open(os.path.join(folder_path,"topic_content.json"), "w") as fp:
    json.dump(topic_content,fp) 


## Assistant Design

### Load data

In [32]:
import json
import os
folder_path ="../../data/agent_assistant"
with open(os.path.join(folder_path, "enriched_emb.json"), "r") as file:
    enriched_emb = json.load(file)
with open(os.path.join(folder_path, "enriched_articles.json"), "r") as file:
    enriched_articles = json.load(file)
with open(os.path.join(folder_path, "topic_content.json"), "r") as file:
    topic_content = json.load(file)

### 1. Tool to generate conversation

In [ ]:
openai.api_version = "2023-05-15"

def generate_conversation(streaming =False):
    user_message =f""" 
    Generate a phone conversation between an customer's employee and an ADP support agent in the area of HR and Payroll. The employee has question and the support agent tried to identity the problems and 
    and take time to use different search tool to come up with answer to the employee.
    Your response:
 
"""
    system_message = """
    You are a customer support agent for ADP company. 

    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ],
        stream=streaming
    )
    return response
            


### 2. Tool to extract problems from conversation. 

In [ ]:
openai.api_version = "2023-05-15"

user_message = ""
def extract_problems(conversation):
    user_message =f""" 
    Follow this on going conversation below and extract problems that each party may need help with and formulate the search query to the knowledge base search tool.
    <<conversattion>>
    {conversation}
    <<conversattion>>
    Output your response in JSON format {{"problem":"summary of problem", "search_query":"content of search query"}}
    There can be more than 1 problem(s)
    Output just JSON, nothing else.
    Your response:
 
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    You are given access to knowledge base search tool to find knowledge needed to find answer to questions. 

    """

    response = openai.ChatCompletion.create(
        timeout=100,
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            
conversation=generate_conversation()['choices'][0]['message']['content']
problems=extract_problems(conversation)
problems

### 3. Tool to find article given a problem

##### Brute Force method

In [31]:
import numpy as np  
openai.api_version = "2022-12-01"

def find_article(question, topk=5):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    best_file_name = None  
      
    # Compute cosine similarity between input vector and each label vector
    cosine_list=[]  
    for chunk_id,chunk_content, vector in chunks_emb:  
        #by default, we use embedding for the entire content of the topic (plus topic descrition).
        # If you you want to use embedding on just topic name and description use this code cosine_sim = cosine_similarity(input_vector, vector[0])
        cosine_sim = cosine_similarity(input_vector, vector) 
        cosine_list.append((chunk_id,chunk_content,cosine_sim ))
    cosine_list.sort(key=lambda x:x[2],reverse=True)
    cosine_list= cosine_list[:topk]
    print(cosine_list)
    cosine_list= cosine_list[:topk]
    best_chunks =[chunk[0] for chunk in cosine_list]
    contents = [chunk[1] for chunk in cosine_list]
    return best_chunks, contents
question = "Why is my paycheck smaller than usual"
find_article(question)

[('article_6###0', " Missing or Late Paychecks\n\nAnother common issue is when an employee does not receive their paycheck on time or doesn't receive it at all. This can be due to a variety of reasons such as technical issues, incorrect bank details, or discrepancies with employee data.\n\nIf an employee misses their paycheck or it is late, they should reach out to their employer or HR department and provide as much information as possible. They should also confirm that their bank details are correct and notify their employer if there are any changes.\n\n3. Incorrect Deductions\n\nEmployees may also experience issues with payroll deductions, where their paycheck does not reflect the correct amount of taxes or any other deductions that were agreed on. This can result in underpaid wages and late fees.\n\nIf an employee realizes that they are being underpaid and has identified that it is due to incorrect deductions, they should contact their payroll administrator to resolve the issue. A p

(['article_6###0',
  'article_0###0',
  'article_0###0',
  'article_7###0',
  'article_7###0'],
 [" Missing or Late Paychecks\n\nAnother common issue is when an employee does not receive their paycheck on time or doesn't receive it at all. This can be due to a variety of reasons such as technical issues, incorrect bank details, or discrepancies with employee data.\n\nIf an employee misses their paycheck or it is late, they should reach out to their employer or HR department and provide as much information as possible. They should also confirm that their bank details are correct and notify their employer if there are any changes.\n\n3. Incorrect Deductions\n\nEmployees may also experience issues with payroll deductions, where their paycheck does not reflect the correct amount of taxes or any other deductions that were agreed on. This can result in underpaid wages and late fees.\n\nIf an employee realizes that they are being underpaid and has identified that it is due to incorrect deduct

In [33]:
import numpy as np  
openai.api_version = "2022-12-01"

def find_article(question, topk=5):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    best_file_name = None  
      
    # Compute cosine similarity between input vector and each label vector
    cosine_list=[]  
    for topic_id, vector in enriched_emb.items():  
        #by default, we use embedding for the entire content of the topic (plus topic descrition).
        # If you you want to use embedding on just topic name and description use this code cosine_sim = cosine_similarity(input_vector, vector[0])
        cosine_sim = cosine_similarity(input_vector, vector[1]) 
        cosine_list.append((topic_id,cosine_sim ))
    cosine_list.sort(key=lambda x:x[1],reverse=True)
    cosine_list= cosine_list[:topk]
    print(cosine_list)
    best_topics = [topic[0] for topic in cosine_list]
    article_files =[best_topic.split("###")[0] for best_topic in best_topics]
    contents = [topic_content[best_topic] for best_topic in best_topics]
    return article_files, contents
question = "Why my paycheck is smaller than usual"
find_article(question)

[('article_6###Incorrect Deductions', 0.816255147684242), ('article_6###Missing or Late Paychecks', 0.8110678496892233), ('article_7###Other Deductions', 0.8045249686543529), ('article_35###Incorrect Payroll Deductions', 0.8022426013113502), ('article_74###Payroll errors', 0.7941619118045048)]


(['article_6', 'article_6', 'article_7', 'article_35', 'article_74'],
 ['Employees may also experience issues with payroll deductions, where their paycheck does not reflect the correct amount of taxes or any other deductions that were agreed on.',
  "Another common issue is when an employee does not receive their paycheck on time or doesn't receive it at all. This can be due to a variety of reasons such as technical issues, incorrect bank details, or discrepancies with employee data.",
  'Your pay stub may also include other deductions such as union dues, garnishments or wage attachments, or additional voluntary contributions to your retirement savings plan.',
  'Employees should review their pay stubs prior to signing to ensure accuracy. If there are any discrepancies in the deductions, the employee should contact their supervisor or HR manager immediately to rectify any inaccuracies.',
  "Incorrect payroll calculations or discrepancies with the employee's pay can cause issues, leadin

#### Vector Searh Lib (Faiss) method

In [28]:
import numpy as np  
import faiss

openai.api_version = "2022-12-01"
#Get the array of embeddings for all articles
topic_ids =[]
emb_vectors = []
for topic_id, vector in enriched_emb.items():  
    #by default, we use embedding for the entire content of the topic (plus topic descrition).
    # If you you want to use embedding on just topic name and description use this code cosine_sim = cosine_similarity(input_vector, vector[0])
    topic_ids.append(topic_id)
    emb_vectors.append(vector[1])
emb_vectors = np.float32(emb_vectors)
faiss.normalize_L2(emb_vectors)

index = faiss.IndexFlatIP(len(vector[1])) 
index.add(emb_vectors)


def find_article_emb_vec(question, topk=3):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    input_vector = np.float32([input_vector])
    faiss.normalize_L2(input_vector)
    d,i = index.search(input_vector, k=topk)
    best_topics = [topic_ids[idx] for idx in i[0]]
    print(d)
    output_contents=[topic_content[best_topic] for best_topic in best_topics]
    article_files =[best_topic.split("###")[0] for best_topic in best_topics]
    return article_files, output_contents
question = "Why is my paycheck  smaller than usual"
find_article_emb_vec(question,5)

[[0.8225051  0.82222325 0.8091605  0.80686766 0.8064399 ]]


KeyError: 'article_6###Incorrect Deductions'

### 4. Tool to answer question from the given problem

In [ ]:
openai.api_version = "2023-03-15-preview"

def answer_assist(problem, search_query):

    articles, contents = find_article(search_query,2)
    articles_contents=""
    for article, content in zip(articles, contents):
        articles_contents += f""" 
        article:{article}
        content: {content}
    """
    articles_contents = f"""
    <<knowledge articles>>
    {articles_contents}
    <<knowledge articles>>
    """
    user_message =f""" 
    problem:{problem}
    {articles_contents}
    Your response:
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    Given the question or problem statement and the knowledge article you have, give the answer.
    Rely solely to the guidance from the article.If the knowlege article is not relavant to the question, say you don't know. Do not make up your answer. 
    Cite the name of the knowledge article as source for your answer.
    Format:
    Answer: your answer to the question
    Sources: [source1, source2]
    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content'], articles_contents

question = "When do I receive my W2 form?"
answer, articles_contents = answer_assist(question,question)
print(answer)
print("---------------content-----------------")

print(articles_contents)

### 5. Put tools all together

In [ ]:
def recommend(conversation):
    i=0
    while (i<5): #handle wrong format output
        problems=extract_problems(conversation)
        try:
            problems=json.loads(problems)
            print("problems", problems)
            for problem in problems:
                answer, articles_contents = answer_assist(problem['problem'],problem['search_query'])
                print(answer)
                print("---------------content-----------------")

                print(articles_contents)
            break

        except Exception as e:
            print(e)
            print("problem parsing json, problems string is ", problems)
            i+=1

        



### One time conversation 

In [ ]:
conversation =generate_conversation()['choices'][0]['message']['content']
print(f"Conversation {conversation}")
recommend(conversation)        

    

### Simulate a running conversation 

In [ ]:
import concurrent
openai.api_version = "2023-05-15"
import time
conversation_pause_duration=2
agent_assist_freq=5
conversation_buffer =[]
def stream_conversation(conversation_buffer, pause_duration=5):
    responses = generate_conversation(True)
    conversation_counter =0
    old_conversation_counter =0
    for response in responses:
        content = response['choices'][0]["delta"].get("content","")
        conversation_buffer.append(content) 
        if "\n"  not in content:
            print(content, end="")
        else:
            conversation_counter+=1
            if conversation_counter > old_conversation_counter+2:
                conversation = "".join(conversation_buffer)
                print("starting recommendation")
                print("conversation: ", conversation)
                recommend(conversation)
                print("ending recommendation")
                old_conversation_counter=conversation_counter



        
stream_conversation(conversation_buffer,conversation_pause_duration)